# Overview

This interactive notebook acts as an productive, ituitive way for the Speckle Imaging Group to plan observing runs. In order to ensure that this tool functions properly, there are a list of **assumptions**:

- Input files will be comma separated (create a fork for it to be tab separated)
- For complex filtering and searching of targets, you must provide your own custom filter in the cells marked below
- For "ZERO" values or "EMPTY" values, please use empty commas ",,"

In [ ]:
# !pip3 install --upgrade pip
# !pip3 install pandas numpy
# !pip3 install astropy astroplan -U
# !pip3 install ipywidgets

from astropy.coordinates import SkyCoord
import astropy
# import astroplan
import ipywidgets
from IPython.display import display, clear_output
from astropy.table import QTable
import astropy.units as u
import numpy as np
import pandas as pd

uploaded = ipywidgets.FileUpload(
    accept='',
    multiple=True
)
uploaded

import ipywidgets as widgets
observed_textbox = widgets.Textarea(
    value='''Example:
1
2
10
15
''',
    placeholder='Type something',
    description='Observed: ',
    disabled=False
)

uploaded

# Step 1: Importation of olist/target files 

## Step 1a: ASCII File Importation 

**Run this cell if it's the first night of the observing run, otherwise, skip to Step 1b**

In [ ]:
# for file, file_values in uploaded.value.items():
#     print(file)
#     print(file_values['content'])

decoded_files = []
for file, file_bytes in uploaded.value.items():
    decoded_files.append(file_bytes['content'].decode())

decoded_entries = []

for decoded_file in decoded_files:
    for line in decoded_file.split("\n"):
        line = ' '.join(line.split())
#         line = line.split("#")
#         try:
#             line = line[0] + "," + line[1]
#         except:
#             line = line[0]
        line = line.split("#")[0] # TODO: For now, just remove everything after the # sign and include again later when needed
        if line.count(",") > 2:
            line = line.split(",")
        else:
            line = line.split(" ")
        
        if line[0] == "":
            continue
            
        try:
            if line[4] == '':
                line[4] = 0.0
        except:
            line.append(0.0)

        try:
            if line[5] == '':
                line[5] = 0.0
        except:
            line.append(0.0)
#         try:
#             if line[7] == '':
#                 line[7] = 0.0
#         except:
#             line.append('')
        decoded_entries.append(line)

print(decoded_entries)

# Backlog Table and Observed Table Construction

In this cell, we created two astropy tables: **backlog_table** and **observed_table**

"backlog_table" will hold all the targets that we have yet to observed. "observed_table" will hold all the targets have have been observed.


In [ ]:
backlog_targets = []
observed_targets = []

# Construction of Backlog table
# backlog_table = QTable(names=("star_id", "queue_id", "ra","dec", "epoch", "proper(ra)", "proper(dec)", "mag", "priority", "pi", "comment"))
backlog_table = QTable(names=("star_id", "ra", "dec", "epoch", "proper(ra)", "proper(dec)", "mag"))
backlog_table["star_id"].dtype = np.dtype("S10")
# backlog_table["queue_id"].dtype = np.dtype("S10")
backlog_table["ra"].dtype = np.dtype("S10")
backlog_table["dec"].dtype = np.dtype("S10")
backlog_table["epoch"].dtype = np.dtype("float32")
backlog_table["proper(ra)"].dtype = np.dtype("float32")
backlog_table["proper(dec)"].dtype = np.dtype("float32")
backlog_table["mag"].dtype = np.dtype("float32")
# backlog_table["priority"].dtype = np.dtype("float32")
# backlog_table["pi"].dtype = np.dtype("S10")
# backlog_table["comment"].dtype = np.dtype("S10")

# Construction of Observed table
# observed_table = QTable(names=("star_id", "queue_id", "ra", "dec", "epoch", "proper(ra)", "proper(dec)", "mag", "priority", "pi", "comment"))
observed_table = QTable(names=("star_id", "ra", "dec", "epoch", "proper(ra)", "proper(dec)", "mag"))
observed_table["star_id"].dtype = np.dtype("S10")
# observed_table["queue_id"].dtype = np.dtype("S10")
observed_table["ra"].dtype = np.dtype("S10")
observed_table["dec"].dtype = np.dtype("S10")
observed_table["epoch"].dtype = np.dtype("float32")
observed_table["proper(ra)"].dtype = np.dtype("float32")
observed_table["proper(dec)"].dtype = np.dtype("float32")
observed_table["mag"].dtype = np.dtype("float32")
# observed_table["priority"].dtype = np.dtype("float32")
# observed_table["pi"].dtype = np.dtype("S10")
# observed_table["comment"].dtype = np.dtype("S10")

# Insert data into the table
for i in range(len(decoded_entries)):
    backlog_table.insert_row(i, decoded_entries[i])

# def filter_ra_by_range(table, key_colnames):
#     print(key_colnames)
#     global x
#     print(x)
#     pass

# def filter_dec_by_range(table, key_colnames):
#     global y
#     print(y)
#     pass

## Step 1b: Resume Observation Run From Previous Night(s)

TODO: Allow prompted files to be inputted rather than a hard coded absolute file path

In [ ]:
backlog_table = QTable.read('.ecsv')
observed_table = QTable.read('.ecsv')

In [ ]:
uploaded = ipywidgets.FileUpload(
    accept='',
    multiple=True
)
uploaded

# Parameter Tuning

In [ ]:
# Try using a enterable box for these parameters
global ra_lower_bound
global ra_upper_bound
global dec_lower_bound
global dec_upper_bound

# Edit these parameters
ra_lower_bound = '00:00:00.0'
ra_upper_bound = '24:00:00.0'
dec_lower_bound = '-90:00:00.0'
dec_upper_bound = '+90:00:00.0'
allowed_priority_bands = []

# Range filter for mag
# Display comments on the table
# Filter by bands as well

# def filter_ra_by_range(table, column):
#     if ra_lower_bound < table[column][] < ra_upper_bound:
#         return True
#     return False

# Edit the parameters of group_by in order to sort by columns of your choice
# Possible sort keys: idx, star_id, ra, dec, epoch, proper(ra), proper(dec), mag
backlog_table = backlog_table.group_by(['ra', 'mag'])

# backlog_table = backlog_table.groups.filter(filter_ra_by_range)
# backlog_table = backlog_table.groups.filter(filter_dec_by_range)

# Table Display

- If a different range, filter, or sort by is needed, rerun "Parameter Tuning" code block
    - Then, re-run "Table Display"

In [ ]:
backlog_table.show_in_notebook(display_length=25)

# Mark as Observed

Mark stars as observed in the below textbox. Remove the example placeholder shown with your own list of 'idx's to mark as observed.

After 

In [ ]:
observed_textbox

## Move targets from textbox to saved observed table

Run the below cell in order to save all of the targets by idx, into a temporary table which will be exported at the end of the night.

**Note:**

- If an entry is not a number or a valid ID of the table, it will be automatically thrown away.**

- Idx numbers are not permanent nor important to a particular row, they are simply used as temporary IDs to mark targets as observed

In [ ]:
textbox_string = observed_textbox.value
textbox_string = textbox_string.strip()
textbox_ids = textbox_string.split('\n')
id_upper_bound = len(backlog_table)
textbox_ids = [s for s in textbox_ids if s.isdigit()]
textbox_ids = map(int, textbox_ids)
textbox_ids = [s for s in textbox_ids if s < id_upper_bound]

for idx in textbox_ids:
    observed_table.add_row(backlog_table[idx])

backlog_table.remove_rows(textbox_ids)

In [ ]:
observed_table.show_in_notebook(display_length=25)

# Undo/Redo

Use these boilerplate cells to manually undo/redo addition and removal of rows between tables

In [ ]:
# Example boilerplate to manually add back a row to backlog_table
backlog_table.add_row(["HR 0047", "00:13:19.4", "-84:59:39", 2000.0, 4.0, 8.0, 5.77])
backlog_table.add_row(["HR 0047", "00:13:19.4", "-84:59:39", 2000.0, 4.0, 8.0, 5.77])
backlog_table.add_row(["HR 0047", "00:13:19.4", "-84:59:39", 2000.0, 4.0, 8.0, 5.77])

In [ ]:
# Example boilerplate to manually add back a row to observed_table
observed_table.add_row(["HR 0047", "00:13:19.4", "-84:59:39", 2000.0, 4.0, 8.0, 5.77])
observed_table.add_row(["HR 0047", "00:13:19.4", "-84:59:39", 2000.0, 4.0, 8.0, 5.77])
observed_table.add_row(["HR 0047", "00:13:19.4", "-84:59:39", 2000.0, 4.0, 8.0, 5.77])

In [ ]:
backlog_table.remove_rows([1, 3, 100, 111])

In [ ]:
observed_table.remove_rows([1, 3, 100, 111])

# Save table state(s) for the night

In order to save the state(s) of the table, we will be serializing both the backlog and observed table. When loading data for the next night, only these saved files will be needed, not the initial target files. Will overr



In [ ]:
import datetime

backlog_table.write('backlog_table_' + str(datetime.datetime.utcnow()) + '.ecsv', overwrite=False, serialize_method='data_mask')
observed_table.write('observed_table' + str(datetime.datetime.utcnow()) + '.ecsv', overwrite=False, serialize_method='data_mask')